# Homework 6  - Logistic Regression without using any libraries. 

## Your Name Here

Student Name: Tyler Smedley

Student UT EID: tws933

---

Partner Name:

Partner UT EID:

---

Date Created: 03/05/2024

Date Last Modified:

---

Totoal Points 20. 



In [7]:
# Standard Headers
# You are welcome to add additional headers here if you wish
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Enable inline mode for matplotlib so that Jupyter displays graphs
%matplotlib inline

# Your allowed to use only the above libraries that are imported. No other libs should be used in this assignment. 

## Heart Dataset 

In this Assignment we will work with some patients dataset. 

We have access to 303 patients data. The features are listed below. 

In [8]:
# Your code here
heart_df = pd.read_csv("Heart.csv")
heart_df['Target'] = heart_df['Target'].map({'Yes':0, 'No': 1})
heart_df

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,Target
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,1
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,0
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,0
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,1
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,typical,110,264,0,0,132,0,1.2,2,0.0,reversable,0
299,68,1,asymptomatic,144,193,1,0,141,0,3.4,2,2.0,reversable,0
300,57,1,asymptomatic,130,131,0,0,115,1,1.2,2,1.0,reversable,0
301,57,0,nontypical,130,236,0,2,174,0,0.0,2,1.0,normal,0


**Age:** The person’s age in years

**Sex:** The person’s sex (1 = male, 0 = female)

**ChestPain:** chest pain type

* Value 0: asymptomatic
* Value 1: atypical angina
* Value 2: non-anginal pain
* Value 3: typical angina

**RestBP:** The person’s resting blood pressure (mm Hg on admission to the hospital)

**Chol:** The person’s cholesterol measurement in mg/dl

**Fbs:** The person’s fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
restecg: resting electrocardiographic results

* Value 0: showing probable or definite left ventricular hypertrophy by Estes’ criteria
* Value 1: normal
* Value 2: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

**RestECG:** The person’s maximum heart rate achieved

**MaxHR:** Exercise induced angina (1 = yes; 0 = no)

**Oldpeak:** ST depression induced by exercise relative to rest (‘ST’ relates to positions on the ECG plot. See more here)

**Slope:** the slope of the peak exercise ST segment — 0: downsloping; 1: flat; 2: upsloping

* 0: downsloping; 
* 1: flat; 
* 2: upsloping

**Ca:** The number of major vessels (0–3)

**Thal:** A blood disorder called thalassemia Value 0: NULL (dropped from the dataset previously

* Value 1: fixed defect (no blood flow in some part of the heart)
* Value 2: normal blood flow
* Value 3: reversible defect (a blood flow is observed but it is not normal)

**Target:** Heart disease (1 = no, 0= yes)

# Task 1 - (4 points)
We want to use logistic regerssion to predict if a patient will have heart problems or not. The column "Target" in our datasets includes data about heart disease. If the patient had heart disease, the patient's "Target" value equals 1. Otherwise, "Target" equals 0.

Prepare your data set for predicting heart disease ("Target" column) by using 3 features:

* Age of the patient (Column **"Age"**)
* Gender of the patient (male or female - Column **"Sex"**)
* Cholestrol level of the patient (Column **"Chol"**) 

Split your data into 80% traning data and 20% test data ***without*** using any libraries other than the ones imported above. You must do it manually.

* Do a maximum of **100 iterations**
* Use a very small learning rate for checking your GD implementation. 
* Your are allowed to use your choice of learning rate, like using 0.0001, 0.001 or 0.01 or 0.1 or higher/lower. 
* **Visualize your error/costs over the iterations with a plot**.
* No need to add an y-intercept in this task. 

(**4 points** - 3 points for code, 1 point for cost visualization)

In [9]:
# Selecting the required features and target
features = heart_df[['Age', 'Sex', 'Chol']]
target = heart_df['Target']

# Split the data into training and testing sets manually (80% training, 20% testing)
np.random.seed(0)  # For reproducibility
indices = np.random.permutation(len(features))

train_size = int(len(features) * 0.8)
test_size = len(features) - train_size

train_indices = indices[:train_size]
test_indices = indices[train_size:]

X_train, X_test = features.iloc[train_indices], features.iloc[test_indices]
y_train, y_test = target.iloc[train_indices], target.iloc[test_indices]

# Display the sizes of the train and test sets to verify the split
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, theta):
    m = len(y)
    h = sigmoid(np.dot(X, theta))
    epsilon = 1e-5  # to avoid log(0)
    cost = (-1/m) * np.sum(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon))
    return cost

def gradient_descent(X, y, theta, learning_rate, iterations):
    m = len(y)
    cost_history = np.zeros(iterations)
    
    for i in range(iterations):
        prediction = sigmoid(np.dot(X, theta))
        theta -= (learning_rate/m) * np.dot(X.T, (prediction - y))
        cost_history[i] = compute_cost(X, y, theta)
    
    return theta, cost_history

# Initialize theta (parameters)
theta = np.zeros(X_train.shape[1])

# Convert DataFrame to NumPy for calculation
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

# Define learning rate and number of iterations
learning_rate = 0.001
iterations = 100

# Perform gradient descent
theta_final, cost_history = gradient_descent(X_train_np, y_train_np, theta, learning_rate, iterations)

# Plotting the cost history
plt.plot(range(1, iterations + 1), cost_history, color='blue')
plt.title('Cost/Error over Iterations')
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.grid(True)
plt.show()

(242, 3) (61, 3) (242,) (61,)


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

# Task 2 - (4 points)

Cacluate the Accuracy, Precision, Recall and F1 score of your logistic regression implementaion on the testing set. 
Print the results. 

You may use equations shown in lecture/slides/examples.

Calcuate the accuracy, precision, recall and F1 score of your logistic regression implementaion on the testing set. 
Print the results. (**4 points**)


In [5]:
def predict(X, theta):
    """
    Predict whether the label is 0 or 1 using learned logistic regression parameters
    """
    probabilities = sigmoid(X.dot(theta))
    return [1 if x >= 0.5 else 0 for x in probabilities]

def calculate_performance_metrics(y_true, y_pred):
    """
    Calculate performance metrics including accuracy, precision, recall, and F1 score
    """
    # Convert lists to numpy arrays for vectorized operations
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    true_negatives = np.sum((y_pred == 0) & (y_true == 0))
    false_positives = np.sum((y_pred == 1) & (y_true == 0))
    false_negatives = np.sum((y_pred == 0) & (y_true == 1))
    
    accuracy = (true_positives + true_negatives) / len(y_true)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return accuracy, precision, recall, f1_score

# Predicting on the test set
y_pred_test = predict(X_test, theta)

# Calculating performance metrics
accuracy, precision, recall, f1_score = calculate_performance_metrics(y_test, y_pred_test)

print(accuracy, precision, recall, f1_score)

0.4918032786885246 0.5263157894736842 0.8823529411764706 0.6593406593406594


# Task 3 - (4 points)

Add a y-intercept and repeat the above tasks. Do you see any differences after adding the y-intercept?

(4 points - 2 for code, 1 for cost visualizaiton, 1 for description.)

In [6]:
# Add your code Here! 

# Task 4 - Implement the Bold Driver   - (4 points)

Implement the bold driver into your gradient descent implementation, which lets us have a dynamic learning rate. Visualize the costs and print the accuracy/etc. metrics as before. Do not use a y-intercept this time.

Add a stop condition that stop the GD when the cost is not changing more than 0.001 between iterations. 
Describe the results. Did you stop earlier than 100 iterations?
(4 points - 2 for code, 1 for cost visualizaiton, 1 for description.)



In [7]:
# Add your code Here! 

# Task 5 - Implement the L2 norm regularization.  - (4 points)



Modify your Cost and gradient to implement the l2 norm regularization. Repeat the steps taken in prior tasks and describe your result.

 * Use a y-intercept.
 * Do a maximum of 100 iterations as before and report your accuracy, precision, recall and F1 score.
 * Optional: You can stop earlier, if the cost is not changing more than 0.001 between iterations.
 * Optional: You can use the bold driver, if you want. But a bold driver is not required to perform L2 norm regularizaiton.

(4 points - 2 for code, 1 for cost visualizaiton, 1 for description.)

In [8]:
# Add your code Here! 